In [1]:
import psycopg2
import os
import pandas as pd

In [10]:
conn = psycopg2.connect(dbname="adsdb", user="adsdb")

In [11]:
cur = conn.cursor()

In [4]:
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'formatted'""")
tables = cur.fetchall()

In [5]:
tables

[('demographiccountry_0_070d_1642868817',),
 ('countrycodes_0_8c41_1642852775',),
 ('notes_0_a67b_1642852775',),
 ('population_0_3b8f_1642852775',),
 ('morticd_7_22b4_1642852775',),
 ('morticd_8_9366_1642852775',),
 ('morticd_9_71c9_1642852775',),
 ('morticd_10_1_f695_1642852776',),
 ('morticd_10_2_ffb1_1642852776',),
 ('morticd_10_3_1c6a_1642852776',),
 ('morticd_10_4_453e_1642852776',),
 ('morticd_10_5_8f17_1642852776',),
 ('countrycodes_0_8c41_1642868814',),
 ('notes_0_a67b_1642868814',),
 ('population_0_3b8f_1642868814',),
 ('morticd_7_22b4_1642868814',),
 ('morticd_8_9366_1642868814',),
 ('morticd_9_71c9_1642868814',),
 ('morticd_10_1_f695_1642868814',),
 ('morticd_10_2_ffb1_1642868815',),
 ('morticd_10_3_1c6a_1642868815',),
 ('morticd_10_4_453e_1642868816',),
 ('morticd_10_5_8f17_1642868816',),
 ('demographic_0_070d_1642868817',),
 ('demographiclabels_0_070d_1642868817',)]

In [30]:
table_versions = dict()

for table in tables:
    name_sha, _ , timestamp = table[0].rpartition("_")
    name, _, sha256 = name_sha.rpartition("_")
    
    print(name, sha256, timestamp)
    
    table_versions[name] = table_versions.get(name, []) + [(int(timestamp), sha256)]

demographiccountry_0 070d 1642868817
countrycodes_0 8c41 1642852775
notes_0 a67b 1642852775
population_0 3b8f 1642852775
morticd_7 22b4 1642852775
morticd_8 9366 1642852775
morticd_9 71c9 1642852775
morticd_10_1 f695 1642852776
morticd_10_2 ffb1 1642852776
morticd_10_3 1c6a 1642852776
morticd_10_4 453e 1642852776
morticd_10_5 8f17 1642852776
countrycodes_0 8c41 1642868814
notes_0 a67b 1642868814
population_0 3b8f 1642868814
morticd_7 22b4 1642868814
morticd_8 9366 1642868814
morticd_9 71c9 1642868814
morticd_10_1 f695 1642868814
morticd_10_2 ffb1 1642868815
morticd_10_3 1c6a 1642868815
morticd_10_4 453e 1642868816
morticd_10_5 8f17 1642868816
demographic_0 070d 1642868817
demographiclabels_0 070d 1642868817


In [33]:
def table_name(name, ts_sha):
    ts, sha = ts_sha
    return "formatted." + "_".join((name, sha, str(ts)))

cur.execute("CREATE SCHEMA IF NOT EXISTS trusted;")

for k, v in table_versions.items():
    # Order by timestamp (newest first)
    v = sorted(v)
    
    new_table = f"trusted.{k}"

    # Load all the newest data into new table
    cur.execute(f'''
    CREATE TABLE  IF NOT EXISTS {new_table} AS 
        TABLE {table_name(k, v[0])};
    ''')

In [34]:
conn.commit()

In [13]:
table_versions

{'demographiccountry_0': [(1642868817, '070d')],
 'countrycodes_0': [(1642852775, '8c41'), (1642868814, '8c41')],
 'notes_0': [(1642852775, 'a67b'), (1642868814, 'a67b')],
 'population_0': [(1642852775, '3b8f'), (1642868814, '3b8f')],
 'morticd_7': [(1642852775, '22b4'), (1642868814, '22b4')],
 'morticd_8': [(1642852775, '9366'), (1642868814, '9366')],
 'morticd_9': [(1642852775, '71c9'), (1642868814, '71c9')],
 'morticd_10_1': [(1642852776, 'f695'), (1642868814, 'f695')],
 'morticd_10_2': [(1642852776, 'ffb1'), (1642868815, 'ffb1')],
 'morticd_10_3': [(1642852776, '1c6a'), (1642868815, '1c6a')],
 'morticd_10_4': [(1642852776, '453e'), (1642868816, '453e')],
 'morticd_10_5': [(1642852776, '8f17'), (1642868816, '8f17')],
 'demographic_0': [(1642868817, '070d')],
 'demographiclabels_0': [(1642868817, '070d')]}

In [22]:
df

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1125,None,None,1955,07A,A000,1,01,08,107822,...,2013.0,2727.0,4126.0,None,None,214,40543,NaN,NaN,NaN
1,1125,None,None,1955,07A,A000,2,01,08,105750,...,1745.0,3465.0,5963.0,None,None,82,39632,NaN,NaN,NaN
2,1125,None,None,1955,07A,A001,1,01,08,1302,...,8.0,6.0,2.0,None,None,1,6,NaN,NaN,NaN
3,1125,None,None,1955,07A,A001,2,01,08,739,...,2.0,1.0,2.0,None,None,0,4,NaN,NaN,NaN
4,1125,None,None,1955,07A,A002,1,01,08,69,...,0.0,0.0,0.0,None,None,0,7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281744,4340,None,None,1960,07B,202,2,02,08,4,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN
281745,4340,None,None,1960,07B,203,1,02,08,13,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN
281746,4340,None,None,1960,07B,203,2,02,08,13,...,0.0,2.0,0.0,None,None,0,0,NaN,NaN,NaN
281747,4340,None,None,1960,07B,205,1,02,08,0,...,0.0,0.0,0.0,None,None,0,0,NaN,NaN,NaN
